## LIBRARIES

In [250]:
import folium
import uuid
import pandas as pd
import numpy as np
import time
import math
from datetime import datetime
from typing import List, Dict, Tuple, Optional
import json
from dataclasses import dataclass
from enum import Enum
from datetime import datetime, timedelta

In [251]:
data = pd.read_csv('../data/cleaned_crop_data.csv')

In [252]:
data.dropna(inplace=True)

In [253]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70348 entries, 0 to 70347
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   SAMPLEID                     70348 non-null  object 
 1   LATITUDE                     70348 non-null  float64
 2   LONGITUDE                    70348 non-null  float64
 3   NDVI                         70348 non-null  float64
 4   PRIMARY_CROP_CLASSIFICATION  70348 non-null  object 
dtypes: float64(3), object(2)
memory usage: 2.7+ MB


In [254]:
data.to_csv('../data/cleaned_crop_data.csv', index=False)

## GLOB VARIABLES

In [255]:
global total_time
total_time = 00 #timedelta(0)

## DroneModel & DroneSpecs Class

In [256]:
class DroneModel(Enum):
    CW_30E = "JOUAV_CW_30E"

@dataclass
class DroneSpecs:
    model: DroneModel
    max_flight_time_minutes: int = 480  # 8 hours
    cruise_speed_kmh: int = 90  # Estimated cruise speed
    mission_time_minutes: int = 5  # Time to complete EXAMINE mission
    cost_usd: int = 30000


## GeoCalculator Class

In [257]:
class GeoCalculator:
    """Handles all geospatial calculations"""
    EARTH_RADIUS_KM = 6371.0
    
    @staticmethod
    def calculate_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """Calculate distance between two points in kilometers using Haversine formula"""
        lat1_rad = math.radians(lat1)
        lon1_rad = math.radians(lon1)
        lat2_rad = math.radians(lat2)
        lon2_rad = math.radians(lon2)
        
        delta_lat = lat2_rad - lat1_rad
        delta_lon = lon2_rad - lon1_rad
        
        a = (math.sin(delta_lat / 2) ** 2 + 
             math.cos(lat1_rad) * math.cos(lat2_rad) * 
             math.sin(delta_lon / 2) ** 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        
        return GeoCalculator.EARTH_RADIUS_KM * c
    
    @staticmethod
    def calculate_flight_time(distance_km: float, speed_kmh: float) -> float:
        """Calculate flight time in minutes"""
        return (distance_km / speed_kmh) * 60

## DRONE CLASS

In [258]:
class Drone:
    def __init__(self, drone_id: str, base: 'DroneBase', specs: DroneSpecs = DroneSpecs(DroneModel.CW_30E)):
        self.drone_id = drone_id
        self.base = base
        self.specs = specs
        self.status = "available"  # available, in_mission, charging, maintenance
        self.assigned_tickets: List['Ticket'] = []
        self.current_location = (base.latitude, base.longitude)
        self.remaining_flight_time = specs.max_flight_time_minutes
        self.mission_start_time: Optional[datetime] = None
        self.total_distance_flown = 0.0
        self.missions_completed = 0

    def can_accept_ticket(self, ticket: 'Ticket') -> Tuple[bool, str]:
        """Check if drone can accept a ticket based on flight time constraints"""
        if self.status != "available":
            return False, f"Drone not available (status: {self.status})"
  
        # Calculate round trip distance
        distance_to_target = GeoCalculator.calculate_distance(
            self.base.latitude, self.base.longitude,
            ticket.latitude, ticket.longitude
        )
        round_trip_distance = distance_to_target * 2
        
        # Calculate required flight time
        flight_time = GeoCalculator.calculate_flight_time(
            round_trip_distance, self.specs.cruise_speed_kmh
        )
        mission_time = self.specs.mission_time_minutes
        total_time_required = flight_time + mission_time

        if total_time_required > self.specs.max_flight_time_minutes:
            return False, f"Mission requires {total_time_required:.1f} min, exceeds max flight time"
   
        return True, "Can accept mission"
    
    """For now, implement single ticket assignment"""
    def assign_tickets(self, tickets: List['Ticket']) -> bool:
        if not tickets:
            return False

        ticket = tickets[0]
        can_accept, reason = self.can_accept_ticket(ticket)
        
        if can_accept:
            self.status = "in_mission"
            #self.assigned_tickets = [ticket]
            self.mission_start_time = datetime.now()
            ticket.assign(self)
            self.assigned_tickets.append(ticket)
            return True
        return False
    
    def complete_mission(self) -> Dict:
        """complete current mission and return statistics"""
        if not self.assigned_tickets:
            return {}
            
        ticket = self.assigned_tickets[0]
        distance_flown = GeoCalculator.calculate_distance(
            self.base.latitude, self.base.longitude,
            ticket.latitude, ticket.longitude
        ) * 2  # Round trip
        
        flight_time = GeoCalculator.calculate_flight_time(
            distance_flown, self.specs.cruise_speed_kmh
        )
        mission_duration = flight_time + self.specs.mission_time_minutes
        mission_stats = {
            "drone_id": self.drone_id,
            "ticket_id": ticket.sample_id,
            "distance_km": distance_flown,
            "flight_time_minutes": flight_time + self.specs.mission_time_minutes,
            "mission_start": self.mission_start_time,
            "mission_end": self.mission_start_time + flight_time + self.specs.mission_time_minutes # datetime.now()
        }

        #total_time += mission_stats["mission_end"] - mission_stats["mission_start"]
        total_time += timedelta(minutes=mission_duration)
        #total_tim = total_time.total_seconds() / 3600:.1f
    
        # Update drone state
        self.status = "available"
        self.total_distance_flown += distance_flown
        self.missions_completed += 1
        self.assigned_tickets = self.assigned_tickets.append(ticket)
        self.mission_start_time = None
        
        return mission_stats


## DroneBase Class

In [259]:
class DroneBase:
    def __init__(self, city_name: str, latitude: float, longitude: float):
        self.city_name = city_name
        self.latitude = latitude
        self.longitude = longitude
        self.drones: List[Drone] = []
        self.operational_hours = (8, 17)  # 9AM to 5PM
        self.tickets_completed = 0
        self.total_revenue = 0.0
        
    def add_drones(self, count: int):
        """Add specified number of drones to base"""
        for i in range(count):
            drone_id = f"{self.city_name[:3].upper()}-{len(self.drones)+1:03d}"
            drone = Drone(drone_id, self)
            self.drones.append(drone)
    
    def get_available_drones(self) -> List[Drone]:
        """Get all available drones during operational hours"""
        current_hour = datetime.now().hour
        if not (self.operational_hours[0] <= current_hour <= self.operational_hours[1]):
            return []  # Base out of work hours
            
        return [d for d in self.drones if d.status == "available"]
    
    def get_best_drone_for_ticket(self, ticket: 'Ticket') -> Optional[Drone]:
        """Get the best available drone for a ticket"""
        available_drones = self.get_available_drones()
        suitable_drones = []
        
        for drone in available_drones:
            can_accept, reason = drone.can_accept_ticket(ticket)
            if can_accept:
                distance = GeoCalculator.calculate_distance(
                    self.latitude, self.longitude,
                    ticket.latitude, ticket.longitude
                )
                suitable_drones.append((drone, distance))
        
        if suitable_drones:
            # Return drone with shortest distance to target
            return min(suitable_drones, key=lambda x: x[1])[0]
        return None

## Ticket Class

In [260]:
class Ticket:
    def __init__(self, sample_id: str, latitude: float, longitude: float, 
                 ndvi: float):
        self.sample_id = sample_id
        self.latitude = latitude
        self.longitude = longitude
        self.ndvi = ndvi
        self.assigned = False
        self.assigned_drone: Optional[Drone] = None
        self.priority = self._calculate_priority()
        self.created_time = datetime.now()
        self.assigned_time: Optional[datetime] = None
        self.completed_time: Optional[datetime] = None
        
    def _calculate_priority(self) -> int:
        """Calculate ticket priority based on NDVI (lower NDVI => higher priority)"""
        if self.ndvi < 0.3:
            return 1  # Critical
        elif self.ndvi < 0.5:
            return 2  # High
        else:
            return 3  # Medium
    
    def assign(self, drone: Drone):
        """Assign ticket to drone"""
        self.assigned = True
        self.assigned_drone = drone
        self.assigned_time = datetime.now()

## Assignement Class

In [261]:
class AssignmentStrategy(Enum):
    
    # these are three strategies that we'll implement
    NEAREST_BASE = "nearest_base" 
    ECONOMIC_OPTIMAL = "economic_optimal"
    PRIORITY_BASED = "priority_based"

class TicketAssignmentSystem:
    def __init__(self, strategy: AssignmentStrategy = AssignmentStrategy.NEAREST_BASE):
        self.strategy = strategy
        self.assignment_history = []
        

        
    def assign_tickets(self, bases: List[DroneBase], tickets: List[Ticket]) -> Dict:
        """Assign tickets to drones based on selected strategy"""
        print("entered assign_tickets....")
        assignments = {"successful": 0, "failed": 0, "details": [], "failed_details" : []}
        
        # Sort tickets by priority (critical first)
        unassigned_tickets = [t for t in tickets if not t.assigned]
        unassigned_tickets.sort(key=lambda x: x.priority) # sort by priority (NDVI)
        
        if unassigned_tickets:
            print("unassigned tickets is full...")
        for ticket in unassigned_tickets:
            if self.strategy == AssignmentStrategy.NEAREST_BASE:
                success = self._assign_nearest_base(ticket, bases)
            elif self.strategy == AssignmentStrategy.ECONOMIC_OPTIMAL:
                success = self._assign_economic_optimal(ticket, bases)
            else:  # PRIORITY_BASED
                success = self._assign_priority_based(ticket, bases)
                
            if success:
                assignments["successful"] += 1
                assignments["details"].append({
                    "ticket_id": ticket.sample_id,
                    "assigned_to": ticket.assigned_drone.drone_id,
                    "base": ticket.assigned_drone.base.city_name
                })
            else:
                assignments["failed"] += 1
                assignments["failed_details"].append({
                    "ticket": ticket,
                    "failed": True,
                })
        
        return assignments
    
    def _assign_nearest_base(self, ticket: Ticket, bases: List[DroneBase]) -> bool:
        """Assign to the nearest base with available drone"""
        base_distances = []
        for base in bases:
            distance = GeoCalculator.calculate_distance(
                base.latitude, base.longitude,
                ticket.latitude, ticket.longitude
            )

            drone = base.get_best_drone_for_ticket(ticket)

            if drone:
                base_distances.append((base, drone, distance))
        
        if base_distances:
            minim = min(base_distances, key=lambda x: x[2])
            _, best_drone, _ = minim
            return best_drone.assign_tickets([ticket])
        return False
    

    def _assign_economic_optimal(self, ticket: Ticket, bases: List[DroneBase]) -> bool:
        """Assign based on economic efficiency: reuse existing drones to minimize capital cost."""
        reusable_drones = []
        new_drones = []
        for base in bases:
            for drone in base.get_available_drones():
                can_accept, _ = drone.can_accept_ticket(ticket)
                if not can_accept:
                    continue

                if len(drone.assigned_tickets) > 0:  # Drone already in use
                    reusable_drones.append((drone, base))
                else:  # Drone not yet used (implies purchase cost)
                    new_drones.append((drone, base))
        print('Reusable Drone Number: ', len(reusable_drones))
        print('New Drone Number: ', len(new_drones))

        # Priority 1: Reuse drones that have already been assigned tickets
        for drone, _ in reusable_drones:
            print('test')
            if drone.assign_tickets([ticket]):
                print(f"✔️ Reusing existing drone {drone.drone_id} to minimize cost.")
                return True

        # Priority 2: Use new drones only if necessary
        for drone, _ in new_drones:
            if drone.assign_tickets([ticket]):
                print(f"🆕 New drone {drone.drone_id} assigned (incurs $30,000 cost).")
                #reusable_drones.append(drone)
                return True

        print("❌ No available drones could be assigned.")
        return False
    



    def _assign_priority_based(self, ticket: Ticket, bases: List[DroneBase]) -> bool:
        """Assign based on ticket priority and base capacity"""
        # for high priority tickets, begin with bases with more available drones
        if ticket.priority == 1:  # Critical
            bases_by_capacity = sorted(bases, 
                                     key=lambda b: len(b.get_available_drones()), 
                                     reverse=True)
        else:
            bases_by_capacity = bases
            
        for base in bases_by_capacity:
            drone = base.get_best_drone_for_ticket(ticket)
            if drone:
                return drone.assign_tickets([ticket])
        return False

## LOAD DATA

In [262]:
def load_drone_bases(filepath: str) -> List[DroneBase]:
    """Load drone bases from CSV file"""
    try:
        # Try to read the CSV file
        df = pd.read_csv(filepath)
        
        # Handle different possible column names
        city_col = None
        lat_col = None
        lng_col = None
        
        for col in df.columns:
            col_upper = col.upper()
            if 'CITY' in col_upper or 'NAME' in col_upper:
                city_col = col
            elif 'LAT' in col_upper and 'LNG' not in col_upper:
                lat_col = col
            elif 'LNG' in col_upper or 'LON' in col_upper:
                lng_col = col
        
        if not all([city_col, lat_col, lng_col]):
            raise ValueError("Could not identify required columns in drone_bases.csv")
        
        bases = []
        for _, row in df.iterrows():
            base = DroneBase(
                city_name=str(row[city_col]),
                latitude=float(row[lat_col]),
                longitude=float(row[lng_col])
            )
            bases.append(base)
        
        print(f"✅ Loaded {len(bases)} drone bases from {filepath}")
        return bases
        
    except FileNotFoundError:
        print(f"❌ Could not find {filepath}, using default bases")
        return None
    except Exception as e:
        print(f"❌ Error loading drone bases: {e}, using default bases")
        return None

def load_crop_targets(filepath: str, ndvi_threshold: float = 0.5) -> List[Ticket]:
    """Load crop targets from CSV file"""
    try:
        df = pd.read_csv(filepath)
        
        # Handle different possible column names
        sample_col = None
        lat_col = None
        lng_col = None
        ndvi_col = None
        
        for col in df.columns:
            col_upper = col.upper()
            if 'SAMPLE' in col_upper or 'ID' in col_upper:
                sample_col = col
            elif 'LAT' in col_upper and 'LNG' not in col_upper:
                lat_col = col
            elif 'LNG' in col_upper or 'LON' in col_upper:
                lng_col = col
            elif 'NDVI' in col_upper:
                ndvi_col = col

        
        if not all([sample_col, lat_col, lng_col, ndvi_col]):
            raise ValueError("Could not identify required columns in crop_targets.csv")
        
        # Filter by NDVI threshold
        # df_filtered = df[df[ndvi_col] < ndvi_threshold]
        i=0
        tickets = []
        for _, row in df.iterrows():
            ticket = Ticket(
                sample_id=i,
                latitude=float(row[lat_col]),
                longitude=float(row[lng_col]),
                ndvi=float(row[ndvi_col]),
            )
            tickets.append(ticket)
            i+=1
        
        print(f"✅ Loaded {len(tickets)} crop targets from {filepath})")
        return tickets
        
    except FileNotFoundError:
        print(f"❌ Could not find {filepath}, using generated tickets")
        return None
    except Exception as e:
        print(f"❌ Error loading crop targets: {e}, using generated tickets")
        return e

## Analytics Engine

In [263]:
# ----- Analytics System -----

class AnalyticsEngine:
    def __init__(self):
        self.simulation_data = []
        self.mission_history = []

    def analyze_scenario(self, bases: List[DroneBase], tickets: List[Ticket], 
                        scenario_name: str) -> Dict:
        """Analyze a specific scenario configuration"""
        total_drones = sum(len(base.drones) for base in bases)
        total_tickets = len([t for t in tickets if not t.assigned])
        assigned_tickets = len([t for t in tickets if t.assigned])

        # Calculate theoretical completion time
        avg_mission_time = total_time # minutes (flight + mission time)
        theoretical_min_time = (total_tickets * avg_mission_time) / total_drones
        
        # Calculate coverage efficiency
        coverage_efficiency = assigned_tickets / len(tickets) * 100
        
        # Calculate economic metrics
        total_drone_cost = total_drones * 30000  # $30k per drone
        
        analysis = {
            "scenario_name": scenario_name,
            "total_bases": len(bases),
            "total_drones": total_drones,
            "drones_per_base": total_drones / len(bases),
            "total_tickets": len(tickets),
            "assigned_tickets": assigned_tickets,
            "coverage_efficiency_percent": coverage_efficiency,
            "theoretical_completion_hours": theoretical_min_time / 60,
            "total_drone_investment_usd": total_drone_cost,
            "base_utilization": self._calculate_base_usage(bases)
        }
        return analysis
    
    def _calculate_base_usage(self, bases: List[DroneBase]) -> Dict[str, float]:
        """calculate usage rate for each base"""
        usage = {}
        for base in bases:
            busy_drones = len([d for d in base.drones if d.missions_completed > 0])
            total_drones = len(base.drones)
            usage[base.city_name] = (busy_drones / total_drones * 100) if total_drones > 0 else 0
        return usage
    
    def compare_scenarios(self, scenarios: List[Dict]) -> Dict:
        """compare multiple scenarios and provide recommendations"""
        comparison = {
            "scenarios": scenarios,
            "best_coverage": max(scenarios, key=lambda x: x["coverage_efficiency_percent"]),
            "most_efficient": min(scenarios, key=lambda x: x["theoretical_completion_hours"]),
            "most_economical": min(scenarios, key=lambda x: x["total_drone_investment_usd"]),
            "recommendations": []
        }
        return comparison

# ----- Visualization Enhancements -----

def create_enhanced_visualization(bases: List[DroneBase], tickets: List[Ticket], 
                                analytics: Dict, filename: str = "enhanced_drone_map.html"):
    """Create enhanced visualization with analytics overlay"""
    philippines_map = folium.Map(location=[12.8797, 121.7740], zoom_start=6)

    # Add base markers with drone count info
    for base in bases:
        available_drones = len(base.get_available_drones())
        busy_drones = len([d for d in base.drones if d.status == "in_mission"])
        
        popup_text = f"""
        <b>{base.city_name} Base</b><br>
        Total Drones: {len(base.drones)}<br>
        Available: {available_drones}<br>
        In Mission: {busy_drones}<br>
        Missions Completed: {base.tickets_completed}
        """
    
        folium.Marker(
            location=[base.latitude, base.longitude],
            popup=folium.Popup(popup_text, max_width=200),
            icon=folium.Icon(color="blue", icon="home")
        ).add_to(philippines_map)
    
    priority_colors = {1: "red", 2: "orange", 3: "yellow"}
    
    for ticket in tickets:
        color = "green" if ticket.assigned else priority_colors.get(ticket.priority, "gray")
        
        popup_text = f"""
        <b>Ticket {ticket.sample_id}</b><br>
        NDVI: {ticket.ndvi:.3f}<br>
        Priority: {ticket.priority}<br>
        Status: {'Assigned' if ticket.assigned else 'Unassigned'}
        """
        
        if ticket.assigned:
            popup_text += f"<br>Assigned to: {ticket.assigned_drone.drone_id}"
        
        folium.Marker(
            location=[ticket.latitude, ticket.longitude],
            popup=folium.Popup(popup_text, max_width=200),
            icon=folium.Icon(color=color, icon="leaf")
        ).add_to(philippines_map)
        
        # Add flight path for assigned tickets
        if ticket.assigned and ticket.assigned_drone:
            drone = ticket.assigned_drone
            base_lat, base_lon = drone.base.latitude, drone.base.longitude
            folium.PolyLine(
                locations=[(base_lat, base_lon), (ticket.latitude, ticket.longitude)],
                color="purple",
                weight=2,
                opacity=0.7,
                popup=f"{drone.drone_id} → {ticket.sample_id}"
            ).add_to(philippines_map)

    # Add analytics info box
    analytics_html = f"""
    <div style="position: fixed; 
                top: 10px; left: 10px; width: 300px; height: 200px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:12px; padding: 10px">
    <h4>Mission Analytics</h4>
    <p><b>Total Bases:</b> {analytics['total_bases']}</p>
    <p><b>Total Drones:</b> {analytics['total_drones']}</p>
    <p><b>Coverage Efficiency:</b> {analytics['coverage_efficiency_percent']:.1f}%</p>
    <p><b>Est. Completion:</b> {analytics['theoretical_completion_hours']:.1f} hours</p>
    <p><b>Investment Cost:</b> ${analytics['total_drone_investment_usd']:,}</p>
    </div>
    """
    philippines_map.get_root().add_child(folium.Element(analytics_html))

    philippines_map.save(filename)
    print(f"📊 Enhanced map saved: {filename}")

## THE SIMULATION CODE

In [ ]:
# ----- Main Simulation Function -----

def run_comprehensive_simulation(bases, tickets, scenarios_to_test, numberdays, scenario_name):
    """Run comprehensive simulation with all enhancements"""
    print("Starting Enhanced Drone Swarm Logistics Simulation...")
    print("=" * 60)
    
    
    analytics = AnalyticsEngine()

    scenario_results = []

    for scenario_config in scenarios_to_test:
        print(f"\n Testing {scenario_config['name']}")
        print("-" * 40)

        # Setup scenario
        if "bases_to_use" in scenario_config:
            # Use only first N bases
            scenario_bases = bases[:scenario_config["bases_to_use"]]
        else:
            scenario_bases = bases.copy()

        # Add drones to bases
        for base in scenario_bases:
            base.drones = []  # Reset drones
            base.add_drones(scenario_config["drones_per_base"])

        # Reset ticket assignments
        for ticket in tickets:
            ticket.assigned = False
            ticket.assigned_drone = None

        # Run assignment
        assignment_system = TicketAssignmentSystem(AssignmentStrategy.NEAREST_BASE)
        assignment_results = assignment_system.assign_tickets(scenario_bases, tickets)

        # Analyze results
        scenario_analysis = analytics.analyze_scenario(scenario_bases, tickets, scenario_config["name"])
        scenario_results.append(scenario_analysis)

        # Print results
        print(f" TOTAL TICKETS in day {numberdays}: {assignment_results['failed']+assignment_results['successful']}")
        print(f"✅ Assigned: {assignment_results['successful']} tickets")
        print(f"❌ Failed: {assignment_results['failed']} tickets")
        print(f"📊 Daily Coverage: {scenario_analysis['coverage_efficiency_percent']:.1f}%")
        #print(f"⏱ Est. completion: {scenario_analysis['theoretical_completion_hours']:.1f} hours")
        
        # Create visualization
        filename = f"scenario_{scenario_config['name'].split()[1].lower()}_day={numberdays}_map.html"
        create_enhanced_visualization(scenario_bases, tickets, scenario_analysis, filename)

        export_comprehensive_results(scenario_results, tickets, numberdays, scenario_name=scenario_name)

    
    #print(f"\n✅ Comprehensive simulation complete!")
    return scenario_results, assignment_results

def export_comprehensive_results(scenario_results: List[Dict], tickets: List[Ticket], day, scenario_name):
    """Export all results to CSV files"""

    df_scenarios = pd.DataFrame(scenario_results)
    df_scenarios.to_csv(f"{scenario_name}/scenario_analysis_{scenario_name}_day_{day}.csv", index=False)
    
    # Detailed ticket analysis
    ticket_data = []
    for ticket in tickets:
        ticket_data.append({
            "sample_id": ticket.sample_id,
            "latitude": ticket.latitude,
            "longitude": ticket.longitude,
            "ndvi": ticket.ndvi,
            "priority": ticket.priority,
            "assigned": ticket.assigned,
            "assigned_drone": ticket.assigned_drone.drone_id if ticket.assigned_drone else None,
            "assigned_base": ticket.assigned_drone.base.city_name if ticket.assigned_drone else None
        })
    
    df_tickets = pd.DataFrame(ticket_data)
    df_tickets.to_csv(f"{scenario_name}/detailed_ticket_analysis_{scenario_name}_day_{day}.csv", index=False)
    
    print("📁 Results exported: scenario_analysis.csv, detailed_ticket_analysis.csv, scenario_comparison.json")

In [265]:
import os
os.makedirs("data_logs", exist_ok=True)

print("\n Loading Data...")
bases = load_drone_bases("../data/drone_bases_test.csv")
tickets = load_crop_targets("../data/crop_data_test.csv", ndvi_threshold=10000)


 Loading Data...
✅ Loaded 199 drone bases from ../data/drone_bases_test.csv
✅ Loaded 2499 crop targets from ../data/crop_data_test.csv)


In [266]:
print(len(tickets))

2499


In [267]:
scenario_1 = [
        {"name": "Scenario 1: Balanced", "drones_per_base": 10}
]

temp_tickets = tickets
var = len(tickets)
numb_days = 0

while var > 0:
    results = run_comprehensive_simulation(bases, temp_tickets, scenario_1, numb_days, "scenario_1")
    temp_tickets = [item['ticket'] for item in results[1]['failed_details']]    
    var = len(temp_tickets)
    numb_days +=1

print('total number of days: ', numb_days)

Starting Enhanced Drone Swarm Logistics Simulation...

 Testing Scenario 1: Balanced
----------------------------------------
entered assign_tickets....
unassigned tickets is full...


 TOTAL TICKETS in day 0: 2499
✅ Assigned: 1990 tickets
❌ Failed: 509 tickets
📊 Daily Coverage: 79.6%
📊 Enhanced map saved: scenario_1:_day=0_map.html
📁 Results exported: scenario_analysis.csv, detailed_ticket_analysis.csv, scenario_comparison.json
Starting Enhanced Drone Swarm Logistics Simulation...

 Testing Scenario 1: Balanced
----------------------------------------
entered assign_tickets....
unassigned tickets is full...
 TOTAL TICKETS in day 1: 509
✅ Assigned: 509 tickets
❌ Failed: 0 tickets
📊 Daily Coverage: 100.0%
📊 Enhanced map saved: scenario_1:_day=1_map.html
📁 Results exported: scenario_analysis.csv, detailed_ticket_analysis.csv, scenario_comparison.json
total number of days:  2


In [268]:
scenario_2 = [
    {"name": "Scenario 2: Few bases, many drones", "bases_to_use": 5, "drones_per_base": 100}
]
temp_tickets = tickets
var = len(tickets)
numb_days = 0

while var > 0:
    results = run_comprehensive_simulation(bases, temp_tickets, scenario_2, numb_days,"scenario_2")
    temp_tickets = [item['ticket'] for item in results[1]['failed_details']]    
    var = len(temp_tickets)
    numb_days +=1

print('total number of days scenario 2: ', numb_days)

Starting Enhanced Drone Swarm Logistics Simulation...

 Testing Scenario 2: Few bases, many drones
----------------------------------------
entered assign_tickets....
unassigned tickets is full...
 TOTAL TICKETS in day 0: 2499
✅ Assigned: 500 tickets
❌ Failed: 1999 tickets
📊 Daily Coverage: 20.0%
📊 Enhanced map saved: scenario_2:_day=0_map.html
📁 Results exported: scenario_analysis.csv, detailed_ticket_analysis.csv, scenario_comparison.json
Starting Enhanced Drone Swarm Logistics Simulation...

 Testing Scenario 2: Few bases, many drones
----------------------------------------
entered assign_tickets....
unassigned tickets is full...
 TOTAL TICKETS in day 1: 1999
✅ Assigned: 500 tickets
❌ Failed: 1499 tickets
📊 Daily Coverage: 25.0%
📊 Enhanced map saved: scenario_2:_day=1_map.html
📁 Results exported: scenario_analysis.csv, detailed_ticket_analysis.csv, scenario_comparison.json
Starting Enhanced Drone Swarm Logistics Simulation...

 Testing Scenario 2: Few bases, many drones
-----------

In [269]:

scenario_3 = [
    {"name": "Scenario 3: Many bases, few drones", "drones_per_base": 3}
]

temp_tickets = tickets
var = len(tickets)
numb_days = 0

while var > 0:
    results = run_comprehensive_simulation(bases, temp_tickets, scenario_3, numb_days, "scenario_3")
    temp_tickets = [item['ticket'] for item in results[1]['failed_details']]    
    var = len(temp_tickets)
    numb_days +=1

print('total number of days scenario 3: ', numb_days)

Starting Enhanced Drone Swarm Logistics Simulation...

 Testing Scenario 3: Many bases, few drones
----------------------------------------
entered assign_tickets....
unassigned tickets is full...
 TOTAL TICKETS in day 0: 2499
✅ Assigned: 597 tickets
❌ Failed: 1902 tickets
📊 Daily Coverage: 23.9%
📊 Enhanced map saved: scenario_3:_day=0_map.html
📁 Results exported: scenario_analysis.csv, detailed_ticket_analysis.csv, scenario_comparison.json
Starting Enhanced Drone Swarm Logistics Simulation...

 Testing Scenario 3: Many bases, few drones
----------------------------------------
entered assign_tickets....
unassigned tickets is full...
 TOTAL TICKETS in day 1: 1902
✅ Assigned: 597 tickets
❌ Failed: 1305 tickets
📊 Daily Coverage: 31.4%
📊 Enhanced map saved: scenario_3:_day=1_map.html
📁 Results exported: scenario_analysis.csv, detailed_ticket_analysis.csv, scenario_comparison.json
Starting Enhanced Drone Swarm Logistics Simulation...

 Testing Scenario 3: Many bases, few drones
-----------